In [14]:
%%time
import xarray as xr
import s3fs
import zarr
from dask.distributed import Client

client = Client()
client.get_versions(check=True)
client

CPU times: user 13.1 ms, sys: 136 µs, total: 13.3 ms
Wall time: 24 ms


### Test 1

* 4 Variables: time, lat, lon, analysed_sst
* 1 dataset
* 3 m4.2xlarge ec2 instances

In [15]:
%%time
sst_netcdf = 'https://podaac-opendap.jpl.nasa.gov:443/opendap/allData/ghrsst/data/GDS2/L4/GLOB/JPL/MUR/v4.1/2002/152/20020601090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc?time[0:1:0],lat[0:1:17998],lon[0:1:35999],analysed_sst[0:1:0][0:1:17998][0:1:35999]'
ds = xr.open_dataset(sst_netcdf, chunks={'time': 366, 'lat': 250, 'lon': 500})
compressor = zarr.Blosc(cname='zstd', clevel=5, shuffle=zarr.Blosc.AUTOSHUFFLE)
encoding = {v: {'compressor': compressor} for v in ds.data_vars}

f_zarr = 'ds-data-projects/mur_sst'
fs = s3fs.S3FileSystem(anon=False)
d = s3fs.S3Map(f_zarr, s3=fs)
ds.to_zarr(store=d, mode='w', encoding=encoding)

CPU times: user 36.5 s, sys: 1.2 s, total: 37.7 s
Wall time: 11min 2s


### Test 2

Change: 2 datasets

* 4 Variables: time, lat, lon, analysed_sst
* 2 datasets
* 3 m4.2xlarge ec2 instances

In [16]:
%%time
sst_netcdf = [
    'https://podaac-opendap.jpl.nasa.gov:443/opendap/allData/ghrsst/data/GDS2/L4/GLOB/JPL/MUR/v4.1/2002/152/20020601090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc?time[0:1:0],lat[0:1:17998],lon[0:1:35999],analysed_sst[0:1:0][0:1:17998][0:1:35999]',
    'https://podaac-opendap.jpl.nasa.gov:443/opendap/allData/ghrsst/data/GDS2/L4/GLOB/JPL/MUR/v4.1/2002/153/20020602090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc?time[0:1:0],lat[0:1:17998],lon[0:1:35999],analysed_sst[0:1:0][0:1:17998][0:1:35999]'
]
ds = xr.open_mfdataset(sst_netcdf, chunks={'time': 366, 'lat': 250, 'lon': 500})
compressor = zarr.Blosc(cname='zstd', clevel=5, shuffle=zarr.Blosc.AUTOSHUFFLE)
encoding = {v: {'compressor': compressor} for v in ds.data_vars}

f_zarr = 'ds-data-projects/mur_sst'
fs = s3fs.S3FileSystem(anon=False)
d = s3fs.S3Map(f_zarr, s3=fs)
ds.to_zarr(store=d, mode='w', encoding=encoding)

CPU times: user 50.5 s, sys: 1.83 s, total: 52.3 s
Wall time: 18min 24s


#### Test 3

Change: 1 dataset, all variables (7 total)

* 7 Variables: time, lat, lon, analysed_sst, analysis_error, mask, sea_ice_fraction
* 1 dataset
* 3 m4.2xlarge ec2 instances

In [18]:
%%time
sst_netcdf = [
    'https://podaac-opendap.jpl.nasa.gov:443/opendap/allData/ghrsst/data/GDS2/L4/GLOB/JPL/MUR/v4.1/2002/152/20020601090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc']
ds = xr.open_mfdataset(sst_netcdf, chunks={'time': 366, 'lat': 250, 'lon': 500})
compressor = zarr.Blosc(cname='zstd', clevel=5, shuffle=zarr.Blosc.AUTOSHUFFLE)
encoding = {v: {'compressor': compressor} for v in ds.data_vars}

f_zarr = 'ds-data-projects/mur_sst'
fs = s3fs.S3FileSystem(anon=False)
d = s3fs.S3Map(f_zarr, s3=fs)
ds.to_zarr(store=d, mode='w', encoding=encoding)

CPU times: user 2min 51s, sys: 5.66 s, total: 2min 57s
Wall time: 45min 29s


#### Test 4

Change: 3 datasets, 4 variables

* 4 Variables: time, lat, lon, analysed_sst
* 3 datasets
* 3 m4.2xlarge ec2 instances

In [ ]:
%%time
sst_netcdf = [
    'https://podaac-opendap.jpl.nasa.gov:443/opendap/allData/ghrsst/data/GDS2/L4/GLOB/JPL/MUR/v4.1/2002/152/20020601090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc?time[0:1:0],lat[0:1:17998],lon[0:1:35999],analysed_sst[0:1:0][0:1:17998][0:1:35999]',
    'https://podaac-opendap.jpl.nasa.gov:443/opendap/allData/ghrsst/data/GDS2/L4/GLOB/JPL/MUR/v4.1/2002/153/20020602090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc?time[0:1:0],lat[0:1:17998],lon[0:1:35999],analysed_sst[0:1:0][0:1:17998][0:1:35999]',
    'https://podaac-opendap.jpl.nasa.gov:443/opendap/allData/ghrsst/data/GDS2/L4/GLOB/JPL/MUR/v4.1/2002/154/20020603090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc?time[0:1:0],lat[0:1:17998],lon[0:1:35999],analysed_sst[0:1:0][0:1:17998][0:1:35999]'
]
ds = xr.open_mfdataset(sst_netcdf, chunks={'time': 366, 'lat': 250, 'lon': 500})
compressor = zarr.Blosc(cname='zstd', clevel=5, shuffle=zarr.Blosc.AUTOSHUFFLE)
encoding = {v: {'compressor': compressor} for v in ds.data_vars}

f_zarr = 'ds-data-projects/mur_sst'
fs = s3fs.S3FileSystem(anon=False)
d = s3fs.S3Map(f_zarr, s3=fs)
ds.to_zarr(store=d, mode='w', encoding=encoding)